In [14]:
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans 

In [15]:
all_weather_data = pd.read_csv("Weather_Data/Openweather_30_year_Honolulu.csv")
all_weather_data.head()

,dt,dt_iso,timezone,city_name,lat,lon,temp,temp_max,wind_speed,wind_deg,wind_gust,rain_1h,rain_3h,clouds_all
0,694224000,1992-01-01 00:00:00 +0000 UTC,-36000,Honolulu,21.309884,-157.85814,76.64,82.85,19.46,70,27.74,NaN,NaN,40
1,694227600,1992-01-01 01:00:00 +0000 UTC,-36000,Honolulu,21.309884,-157.85814,74.21,76.96,13.87,50,NaN,NaN,NaN,20
2,694231200,1992-01-01 02:00:00 +0000 UTC,-36000,Honolulu,21.309884,-157.85814,73.67,76.69,11.41,50,NaN,NaN,NaN,75
3,694234800,1992-01-01 03:00:00 +0000 UTC,-36000,Honolulu,21.309884,-157.85814,73.81,76.91,13.87,70,NaN,NaN,NaN,75
4,694238400,1992-01-01 04:00:00 +0000 UTC,-36000,Honolulu,21.309884,-157.85814,72.45,75.83,8.05,60,NaN,NaN,NaN,75


In [16]:
weather_data = all_weather_data[["dt_iso", "temp", "temp_max", "wind_speed", "rain_1h", "clouds_all"]]
weather_data['dt_iso'] = [str(x).replace('+0000 UTC', '') for x in weather_data['dt_iso']]
weather_data['dt_iso'] = pd.to_datetime(weather_data['dt_iso'])
weather_data = weather_data.fillna(0)
weather_data.head()

C:\Users\jonm5\AppData\Local\Temp\ipykernel_25992\1805575280.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_data['dt_iso'] = [str(x).replace('+0000 UTC', '') for x in weather_data['dt_iso']]
C:\Users\jonm5\AppData\Local\Temp\ipykernel_25992\1805575280.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_data['dt_iso'] = pd.to_datetime(weather_data['dt_iso'])


,dt_iso,temp,temp_max,wind_speed,rain_1h,clouds_all
0,1992-01-01 00:00:00,76.64,82.85,19.46,0.0,40
1,1992-01-01 01:00:00,74.21,76.96,13.87,0.0,20
2,1992-01-01 02:00:00,73.67,76.69,11.41,0.0,75
3,1992-01-01 03:00:00,73.81,76.91,13.87,0.0,75
4,1992-01-01 04:00:00,72.45,75.83,8.05,0.0,75


In [17]:
weather_trends = weather_data.groupby(pd.Grouper(key='dt_iso',axis=1, freq="D")).mean()
weather_trends = weather_trends.reset_index()
weather_trends.head()

,dt_iso,temp,temp_max,wind_speed,rain_1h,clouds_all
0,1992-01-01,73.460833,76.243750,13.034167,0.000000,53.083333
1,1992-01-02,73.349583,77.503750,11.861250,0.065833,40.833333
2,1992-01-03,74.239167,76.031667,12.951250,0.013750,39.541667
3,1992-01-04,73.683333,76.971667,8.130833,0.006250,29.208333
4,1992-01-05,72.560417,74.882917,4.702083,0.000000,24.833333


In [18]:
historical_dates = weather_trends["dt_iso"]

In [19]:
historical_trends = weather_trends[["temp", "temp_max", "wind_speed", "rain_1h", "clouds_all"]]

In [20]:
k = list(range(1,7))
inertia=[]
for i in k:
    model = KMeans(n_clusters=i, random_state=0)
    model.fit(historical_trends)
    inertia.append(model.inertia_)

elbow_data = {
    "k": k,
    "inertia": inertia
}

df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

:Curve   [k]   (inertia)

In [21]:
weather_model = KMeans(n_clusters=4)

In [22]:
weather_model.fit(historical_trends)

KMeans(n_clusters=4)

In [23]:
weather_segments = weather_model.predict(historical_trends)

In [24]:
df_historical_trends = historical_trends.copy()
df_historical_trends["Weather Segment"] = weather_segments
df_historical_trends.head()

,temp,temp_max,wind_speed,rain_1h,clouds_all,Weather Segment
0,73.460833,76.243750,13.034167,0.000000,53.083333,0
1,73.349583,77.503750,11.861250,0.065833,40.833333,1
2,74.239167,76.031667,12.951250,0.013750,39.541667,1
3,73.683333,76.971667,8.130833,0.006250,29.208333,1
4,72.560417,74.882917,4.702083,0.000000,24.833333,3


In [25]:
df_historical_trends["Date"] = historical_dates
df_historical_trends = df_historical_trends.set_index("Date")
df_historical_trends.head()

,temp,temp_max,wind_speed,rain_1h,clouds_all,Weather Segment
Date,,,,,,
1992-01-01,73.460833,76.243750,13.034167,0.000000,53.083333,0
1992-01-02,73.349583,77.503750,11.861250,0.065833,40.833333,1
1992-01-03,74.239167,76.031667,12.951250,0.013750,39.541667,1
1992-01-04,73.683333,76.971667,8.130833,0.006250,29.208333,1
1992-01-05,72.560417,74.882917,4.702083,0.000000,24.833333,3


In [34]:
averages = df_historical_trends.groupby([df_historical_trends.index.month, df_historical_trends.index.day]).mean()
averages = averages.drop(['temp', 'rain_1h', 'Weather Segment'], axis=1)
averages.head(50)

temp_max  wind_speed  clouds_all
Date Date                                   
1    1     74.791667    9.201169   47.747312
     2     74.460497    8.603024   47.004032
     3     74.412043    9.702500   43.030914
     4     74.984301   10.580887   43.646505
     5     74.957997   10.644691   40.676075
     6     74.527728   10.860632   39.809140
     7     74.413266   10.020390   34.088710
     8     74.110470    8.893387   33.795699
     9     74.404395    8.344126   39.189516
     10    74.820309    9.150699   36.264785
     11    74.686868    8.689462   35.848118
     12    74.302298    7.994261   38.510753
     13    74.557379    8.540806   37.637097
     14    74.611237    9.098804   43.005376
     15    75.144220    9.923535   41.352151
     16    75.014435    9.875726   41.538978
     17    74.366734   10.545108   41.670699
     18    74.349476   10.883481   41.145161
     19    74.551102   10.547137   40.341398
     20    74.918226   10.447177   44.475806
     21    74.621694   10.074583   40.614247
     22    74.757513   10.967339   39.879032
     23    75.074247   10.585470   39.205645
     24    74.633320    9.305605   39.901882
     25    74.406290    9.107728   41.590054
     26    74.912849   10.087540   47.262097
     27    74.697540    9.774355   43.259409
     28    74.763280   10.587567   43.020161
     29    74.942997   11.512755   45.614247
     30    74.348562   10.416223   39.095430
     31    74.463091   10.704503   38.571237
2    1     74.968239   11.118831   42.287634
     2     74.370228    9.916599   39.571237
     3     74.070699   10.828548   37.801075
     4     73.503871   10.283817   38.776882
     5     74.078562   10.867137   40.748656
     6     74.441183   10.287339   41.651882
     7     73.788105   10.204597   37.975806
     8     73.374758   10.064758   37.518817
     9     73.693683   10.453239   35.966398
     10    73.912164   10.220497   36.805108
     11    73.908293   10.584651   39.600806
     12    73.716183    9.375766   42.470430
     13    74.484543   10.186882   44.633065
     14    74.639960   11.162554   50.329301
     15    74.493051   10.867110   46.536290
     16    74.462903   11.196384   41.477151
     17    74.766573   11.786868   45.580645
     18    75.000390   11.060215   48.205645
     19    74.773710   10.532702   43.736559

In [35]:
totals = df_historical_trends.groupby([df_historical_trends.index.month, df_historical_trends.index.day]).sum()
totals = totals.drop(['temp', 'temp_max', 'wind_speed', 'Weather Segment', 'clouds_all'], axis=1)
totals.head(50)

rain_1h
Date Date          
1    1     6.419583
     2     6.303750
     3     9.831667
     4     1.855417
     5     1.654167
     6     2.097917
     7     0.406250
     8     0.403333
     9     2.895000
     10    1.629583
     11    2.199167
     12    1.473333
     13    2.660000
     14    1.770417
     15    2.098333
     16    1.380000
     17    0.855417
     18    2.504583
     19    2.570000
     20    5.444583
     21    0.915833
     22    1.175000
     23    2.058750
     24    1.646250
     25    2.575000
     26    5.460000
     27    1.517500
     28    2.759167
     29    4.073750
     30    4.292083
     31    2.192500
2    1     1.731667
     2     2.595417
     3     2.405000
     4     1.862917
     5     2.440417
     6     2.202917
     7     2.545833
     8     1.238333
     9     0.882917
     10    0.945833
     11    3.218750
     12    1.799583
     13    1.732500
     14    9.376667
     15    2.311250
     16    2.247500
     17    1.972083
     18    1.439167
     19    2.549583